# Olist E-commerce Performance & Profitability Analysis  
## Notebook 01: Data Cleaning & Integration

### Objective
This notebook focuses on the **data understanding, inspection, cleaning, and integration** of all Olist datasets.  
The goal is to create one **clean, consistent, and analysis-ready master table** that combines data from all sources.

### Business Context
Olist is a Brazilian e-commerce marketplace connecting small businesses (sellers) to customers nationwide.  
However, the company has faced **declining customer satisfaction** and **inconsistent profitability** across categories.

To help the business recover, I aim to:
1. Identify the **key causes of customer dissatisfaction** (e.g., delays, product issues).
2. Find the **drivers of low profitability** (e.g., high shipping cost, poor-performing sellers).
3. Prepare clean data for **visual and dashboard analysis** in the next phase.

### This Notebook Covers:
1. Data Loading — importing all 9 CSV files  
2. Initial Inspection — overview of structure, size, and missing values  
3. Data Cleaning — handling missing values, date formatting, category mapping  
4. Data Integration — merging multiple tables into one “master” dataset  
5. Export — saving the cleaned dataset for the next analysis notebook

## Step 1: Import Required Libraries
I’ll import all the libraries required for data loading, cleaning, and merging.

In [172]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore')

## Dataset Overview: Olist Data Dictionary

| File Name | Primary Purpose | Foreign Keys (for joining) |
| :--- | :--- | :--- |
| **`olist_orders_dataset.csv`** | **Master Order Table** (Status, Dates, Customer ID) | `customer_id` |
| **`olist_customers_dataset.csv`** | Customer location and unique user ID. | `customer_id` (from Orders) |
| **`olist_geolocation_dataset.csv`** | Geographical coordinates for zip codes. | `geolocation_zip_code_prefix` (from Customers/Sellers) |
| **`olist_order_items_dataset.csv`** | Transactional core (price, seller, product). | `order_id`, `product_id`, `seller_id` |
| **`olist_order_payments_dataset.csv`** | Payment method, installments, and total value. | `order_id` |
| **`olist_order_reviews_dataset.csv`** | Customer satisfaction score (1-5) and message. | `order_id` |
| **`olist_products_dataset.csv`** | Physical details, attributes, and category name. | `product_id` (from Order Items) |
| **`olist_sellers_dataset.csv`** | Seller registration and location information. | `seller_id` (from Order Items) |
| **`product_category_name_translation.csv`** | Look-up table to convert Portuguese names to English. | `product_category_name` (from Products) |

---

### Loading all files

In [20]:
orders_data = pd.read_csv('../raw_data/olist_orders_dataset.csv')
customers_data = pd.read_csv('../raw_data/olist_customers_dataset.csv')
geolocation_data = pd.read_csv('../raw_data/olist_geolocation_dataset.csv')
orders_item_data = pd.read_csv('../raw_data/olist_order_items_dataset.csv')
orders_payment_data = pd.read_csv('../raw_data/olist_order_payments_dataset.csv')
orders_review_data = pd.read_csv('../raw_data/olist_order_reviews_dataset.csv')
products_data = pd.read_csv('../raw_data/olist_products_dataset.csv')
sellers_data = pd.read_csv('../raw_data/olist_sellers_dataset.csv')
category_names_data = pd.read_csv('../raw_data/product_category_name_translation.csv')

In [21]:
def inspect(df, df_name='df'):
    print(df_name)
    print(f"\nShape: {df.shape}")
    print("\nDtypes & Null Counts:")

    df.info()
    print(f"\nData Preview: {df.head()}")
    print(f"\nNull Count: {df.isnull().sum()}\n")

In [22]:
data_objects_map = {
    'Orders Data': orders_data,
    'Customers Data': customers_data,
    'GeoLocation Data': geolocation_data,
    'Orders Items Data': orders_item_data,
    'Orders Payments Data': orders_payment_data,
    'Orders Reviews Data': orders_review_data,
    'Products Data': products_data,
    'Sellers Data': sellers_data,
    'Category Names Data': category_names_data
}

for name, data in data_objects_map.items():
    inspect(data, name)

Orders Data

Shape: (99441, 8)

Dtypes & Null Counts:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB

Data Preview:                            order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb474

## Initial Data Inspection Summary

### olist_orders_dataset

This is the main **orders timeline** table ($\approx 99.4k$ records). All date columns are currently stored as `object` (string) and **must be converted** to datetime objects for analysis. Approximately **3% of orders lack a customer delivery date** and $\approx 1.8\%$ lack a carrier dispatch date, strongly indicating **canceled or undelivered orders**.

### olist_customers_dataset

This dataset is clean and complete ($\approx 99.4k$ records with 0 nulls). The critical feature is the **`customer_unique_id`**, which is necessary for distinguishing between one-time buyers and repeat customers for RFM analysis.

### olist_geolocation_dataset

This table is **very large** ($\approx 1$ million records) and fully complete. Given its size, I must confirm that I **only load the required zip codes** during the merge phase to avoid memory issues, even after optimizing its high memory usage (38+ MB).

### olist_order_items_dataset

This transactional table is complete and contains **$\approx 112.6k$ item records** linked to $\approx 99.4k$ orders. Since it contains multiple entries per `order_id` (for multiple items), I must be careful to aggregate prices and freight values correctly when merging.

### olist_order_payments_dataset

This table has $\approx 103.8k$ records, slightly more than the number of orders, confirming that some **orders were paid for using multiple payment types** (e.g., credit card and voucher) or sequential transactions. No critical missing values were observed.

### olist_order_reviews_dataset

This table has major sparsity issues in its qualitative data. While review scores are complete, **$\approx 88\%$ of reviews lack a comment title** and $\approx 59\%$ lack a message. I can only rely on the **`review_score`** for satisfaction analysis unless advanced text analysis is performed on the available messages.

### olist_products_dataset

This catalog of $\approx 32.9k$ unique products is relatively clean. The main issue is that $\approx 2\%$ of records lack product attribute data (category, length, weight), which means these **products cannot be categorized or used in logistics analysis** (freight cost estimation).

### olist_sellers_dataset

This small table ($\approx 3k$ records) is fully complete and will be critical for linking seller performance metrics (derived from **Order Items**) to their geographical location.

### product_category_name_translation.csv

This small, complete table ($\approx 71$ records) is essential for converting the Portuguese category names into **English** for immediate, human-readable reporting.

## Keys finding

In [26]:
for name, data in data_objects_map.items():
    print(name)
    print(data.nunique())
    print('\n')

Orders Data
order_id                         99441
customer_id                      99441
order_status                         8
order_purchase_timestamp         98875
order_approved_at                90733
order_delivered_carrier_date     81018
order_delivered_customer_date    95664
order_estimated_delivery_date      459
dtype: int64


Customers Data
customer_id                 99441
customer_unique_id          96096
customer_zip_code_prefix    14994
customer_city                4119
customer_state                 27
dtype: int64


GeoLocation Data
geolocation_zip_code_prefix     19015
geolocation_lat                717360
geolocation_lng                717613
geolocation_city                 8011
geolocation_state                  27
dtype: int64


Orders Items Data
order_id               98666
order_item_id             21
product_id             32951
seller_id               3095
shipping_limit_date    93318
price                   5968
freight_value           6999
dtype: int64


Ord

## Count of Items in keys

In [41]:
orders_payment_data

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45
...,...,...,...,...,...
103881,0406037ad97740d563a178ecc7a2075c,1,boleto,1,363.31
103882,7b905861d7c825891d6347454ea7863f,1,credit_card,2,96.80
103883,32609bbb3dd69b3c066a6860554a77bf,1,credit_card,1,47.77
103884,b8b61059626efa996a60be9bb9320e10,1,credit_card,5,369.54


In [48]:
print("Orders Data:")
print("Orders ID and Customer ID Unique Count:")
display(orders_data['order_id'].nunique())
display(orders_data['customer_id'].nunique())

print("\nCustomers Data:")
print("Customer ID and Customer Unique ID Unique count:")
display(customers_data['customer_id'].nunique())
display(customers_data['customer_unique_id'].nunique())

print("\nProducts Data:")
print("Product ID Unique count:")
display(products_data['product_id'].nunique())

print("\nOrders Item Data:")
print("Order ID and Product ID Unique count:")
display(orders_item_data['order_id'].nunique())
display(orders_item_data['product_id'].nunique())

print('\nOrders Review Data:')
print('Order ID Unique count:')
display(orders_review_data['order_id'].nunique())

print("\nOrders Payment Data:")
print('Order ID Unique count:')
display(orders_payment_data['order_id'].nunique())

Orders Data:
Orders ID and Customer ID Unique Count:


99441

99441


Customers Data:
Customer ID and Customer Unique ID Unique count:


99441

...

32951


Orders Review Data:
Order ID Unique count:


98673


Orders Payment Data:
Order ID Unique count:


99440

## Data Model and Relational Integrity Check

### Primary Keys (PKs)

I successfully identified the primary identifier for each core entity table:

| Table | Primary Key (PK) |
| :--- | :--- |
| **Orders Data** | `order_id` |
| **Customers Data** | `customer_unique_id` |
| **Reviews Data** | `review_id` |
| **Products Data** | `product_id` |
| **Sellers Data** | `seller_id` |

### Inter-Table Relationships (Foreign Keys)

The transactional nature of the data requires three main merging flows:

| Data Set | Link Key (Foreign Key) | Target Table | Purpose |
| :--- | :--- | :--- | :--- |
| **Orders Items Data** | `order_id` | Orders Data | Links items back to the parent order and customer. |
| | `product_id` | Products Data | Links items to product attributes (category, weight). |
| | `seller_id` | Sellers Data | Identifies the seller responsible for fulfillment. |
| **Orders Reviews Data** | `order_id` | Orders Data | Links satisfaction score to the order timeline. |
| **Orders Data** | `customer_id` | Customers Data | Links the order to the customer's unique ID. |
| **GeoLocation Data** | *None* | *None* | Merging requires matching zip code prefixes, not a unique transaction key. |

### Integrity and Cardinality Observations

Deep inspection revealed key structural differences that must be handled during the merging phase:

1.  **Customer De-duplication:** The `Customers Data` has **99,441** `customer_id` entries (one per order) but only **96,096** **`customer_unique_id`**. This confirms $\approx 3,345$ **repeat buyers**, which is crucial for RFM analysis.

2.  **Missing Order Details:** I have **99,441** orders in the `Orders Data` table, but the child tables report slightly fewer:
    * `Orders Items Data`: **98,666** unique `order_id`s ($\approx 0.8\%$ missing).
    * `Orders Reviews Data`: **98,673** unique `order_id`s.
    These slight gaps suggest a few orders were placed but had **no items** or **no review entry** generated. I should prioritize using the `Orders Data` as the master list and performing **left joins** to retain all original orders.

3.  **Order Item Duplication:** The `Orders Items Data` has $\approx 112.6k$ total rows but only **98,666** unique `order_id`s, confirming that **multiple items often belong to a single order**. This is a normal one-to-many relationship, requiring careful aggregation (summing `price` and `freight_value`) before joining.

4.  **Geolocation Strategy:** The GeoLocation data cannot be joined directly using a single foreign key. It is a large, descriptive table ($\approx 1$ million rows) that must be merged using the **zip code prefix** (`customer_zip_code_prefix` or `seller_zip_code_prefix`).

## Memory Optimization
### Must
1. `olist_geolocation_dataset`
    - This is taking 38+ MB for now, as it has 1 million records.
    - But I can convert `city` and `state` into category dtype.

In [52]:
# converting to category dtype
geolocation_data[['geolocation_city', 'geolocation_state']] = geolocation_data[['geolocation_city', 'geolocation_state']].astype('category')

## Check Memory usage
memory = round(geolocation_data.memory_usage(deep=True).sum() / 1024**2, 2)
print("Memory Usage After Optimization:", memory,'MB')

Memory Usage After Optimization: 26.61 MB


## Other Tables Memory Optimization

### Memory usage Before Any Optimization

In [56]:
for name, data in data_objects_map.items():
    if name == 'GeoLocation Data':
        continue
    memory = round(data.memory_usage(deep=True).sum() / 1024**2, 2)
    print(name)
    print("Memory Usage:", memory,'MB\n')

Orders Data
Memory Usage: 58.97 MB

Customers Data
Memory Usage: 29.62 MB

Orders Items Data
Memory Usage: 39.43 MB

Orders Payments Data
Memory Usage: 17.81 MB

Orders Reviews Data
Memory Usage: 42.75 MB

Products Data
Memory Usage: 6.79 MB

Sellers Data
Memory Usage: 0.66 MB

Category Names Data
Memory Usage: 0.01 MB



## The Shock:
- After checking with `memory_usage(deep=True)` memory usage increase. Because it shows real total memory used by that table.

In [62]:
def optimize_memory(df, cols, convert_to='category'):
    df[cols] = df[cols].astype(convert_to)

    ## Check Memory usage
    memory = round(df.memory_usage(deep=True).sum() / 1024**2, 2)
    print("Memory Usage After Optimization:", memory,'MB')

In [64]:
## Let's convert order_status into category dtype
optimize_memory(orders_data, 'order_status')

Memory Usage After Optimization: 52.81 MB


In [66]:
## same with customrs data
optimize_memory(customers_data, ['customer_city', 'customer_state'])

Memory Usage After Optimization: 18.32 MB


In [69]:
## Now, orders items data
optimize_memory(orders_item_data, ['price', 'freight_value'], convert_to='float32')

Memory Usage After Optimization: 38.57 MB


### Orders Payment Data
- It has some columns which can be converted to other types for optimization.
- `payment_sequential` is int type and it has values range from 1-29 and can be converted into shorter int type like int8 or int16.
- `payment_type` is object type and has 5 categories and can be converted into category dtype to optimize memory.
- `payment_installments` is int type but can be reduced into smaller int types.
- `payment_value` is float type but can be converted into float32 which is more memory efficient.

In [90]:
orders_payment_data[['payment_sequential', 'payment_installments']] = orders_payment_data[['payment_sequential', 'payment_installments']].astype('int8')
orders_payment_data['payment_type'] = orders_payment_data['payment_type'].astype('category')
orders_payment_data['payment_value'] = orders_payment_data['payment_value'].astype('float32')

## lets check memory
memory = round(orders_payment_data.memory_usage(deep=True).sum() / 1024**2, 2)
print("Memory Usage After Optimization:", memory,'MB')

Memory Usage After Optimization: 9.51 MB


In [93]:
## review_score is better with int8
optimize_memory(orders_review_data, 'review_score', convert_to='int8')

Memory Usage After Optimization: 42.08 MB


In [105]:
products_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  Int8   
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  Int8   
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  Int8   
 7   product_height_cm           32949 non-null  Int8   
 8   product_width_cm            32949 non-null  Int8   
dtypes: Int8(5), float64(2), object(2)
memory usage: 1.3+ MB


In [107]:
## lets reduce some memory by converting to lower data types

optimize_memory(products_data, ['product_name_lenght', 'product_photos_qty', 'product_length_cm', 'product_height_cm', 'product_width_cm'], convert_to='UInt8')
# these 2 will reduce some more memory

optimize_memory(products_data, ['product_description_lenght', 'product_weight_g'], convert_to='UInt16')

Memory Usage After Optimization: 5.54 MB
Memory Usage After Optimization: 5.54 MB


## After Optimization

In [108]:
for name, data in data_objects_map.items():
    memory = round(data.memory_usage(deep=True).sum() / 1024**2, 2)
    print(name)
    print("Memory Usage:", memory,'MB\n')

Orders Data
Memory Usage: 52.81 MB

Customers Data
Memory Usage: 18.32 MB

GeoLocation Data
Memory Usage: 26.61 MB

Orders Items Data
Memory Usage: 38.57 MB

Orders Payments Data
Memory Usage: 9.51 MB

Orders Reviews Data
Memory Usage: 42.08 MB

Products Data
Memory Usage: 5.54 MB

Sellers Data
Memory Usage: 0.66 MB

Category Names Data
Memory Usage: 0.01 MB



## Data Type Optimization (Phase 1)

My primary goal was to address the significant memory usage (over 200 MB) identified during inspection, which was primarily caused by `object` (string) columns being loaded with `deep=True`. I performed an initial optimization pass by converting data types *before* merging.

My optimization strategy involved three main actions:

* **High-Cardinality Strings:** I converted columns with high repetition (e.g., `order_status`, `customer_city`, `payment_type`) from `object` to the `category` dtype. This was the source of the largest memory savings.
* **Numerical Precision:** I downcast numerical columns where `float64` precision was unnecessary (e.g., `price`, `payment_value` to `float32`) and integers to smaller types (e.g., `payment_installments` to `int8`).
* **Handling NaNs in Integers:** For the `products_data`, I used `UInt8` and `UInt16` to efficiently store small positive numbers while preserving `NaN` values, which is not possible with standard `int` types.

### Final Memory Usage After Optimization

This initial pass has significantly reduced the total memory footprint.

* **Orders Data:** 52.81 MB
* **Customers Data:** 18.32 MB
* **GeoLocation Data:** 26.61 MB
* **Orders Items Data:** 38.57 MB
* **Orders Payments Data:** 9.51 MB
* **Orders Reviews Data:** 42.08 MB
* **Products Data:** 5.54 MB
* **Sellers Data:** 0.66 MB
* **Category Names Data:** 0.01 MB

I will reduce this memory footprint further in the next cleaning phase by dropping columns that are not relevant to the business questions (e.g., `review_comment_title`, `review_comment_message`).

# Data Cleaning

### Cleaning Date Columns

In [116]:
date_cols = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']

## converting to datetime
for col in date_cols:
    orders_data[col] = pd.to_datetime(orders_data[col], errors='coerce')

In [117]:
orders_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  category      
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: category(1), datetime64[ns](5), object(2)
memory usage: 5.4+ MB


### Let's veryfiy some patterns

In [131]:
orders_data[orders_data['order_approved_at'].isnull()]['order_status'].unique()

['canceled', 'delivered', 'created']
Categories (8, object): ['approved', 'canceled', 'created', 'delivered', 'invoiced', 'processing', 'shipped', 'unavailable']

- I expect where approved orders are Null, it means user canceled the order, but it has 3 categories `canceled`, `delivered` and `created`.

In [132]:
orders_data[orders_data['order_delivered_carrier_date'].isnull()]['order_status'].unique()

['invoiced', 'processing', 'unavailable', 'canceled', 'created', 'approved', 'delivered']
Categories (8, object): ['approved', 'canceled', 'created', 'delivered', 'invoiced', 'processing', 'shipped', 'unavailable']

- I expect same here but it has 7 categories which needs deeper exploration to know better.

In [133]:
orders_data[orders_data['order_delivered_customer_date'].isnull()]['order_status'].unique()

['invoiced', 'shipped', 'processing', 'unavailable', 'canceled', 'delivered', 'created', 'approved']
Categories (8, object): ['approved', 'canceled', 'created', 'delivered', 'invoiced', 'processing', 'shipped', 'unavailable']

- It also shows that It doens't just mean that customer canceled the order, there are many other causes. Like the product not delivered etc.

### Memory Reduction

In [134]:
memory = round(orders_data.memory_usage(deep=True).sum() / 1024**2, 2)
print("Memory Usage:", memory,'MB\n')

Memory Usage: 20.77 MB



## Data Cleaning: Timestamp Conversion (Orders Table)

### Action Taken

I converted the five `object` type timestamp columns to the `datetime64[ns]` dtype. This was a critical step for both memory optimization and enabling time-based calculations.

* **Result:** This single conversion reduced the `orders_data` memory footprint from **52.81 MB** to **20.77 MB**, a reduction of $\approx 60\%$.

### Key Findings on Null Timestamps

The null values in the date columns are not random errors; they directly correlate with the `order_status` and map to the real-world order fulfillment funnel.

1.  **`order_purchase_timestamp` & `order_estimated_delivery_date` (0 Nulls):**
    These columns are 100% complete. This is logical, as an order cannot exist without a purchase time, and an estimate is generated automatically.

2.  **`order_approved_at` (160 Nulls):**
    These nulls are mostly associated with the `created` and `canceled` statuses, indicating orders that were placed but never confirmed for payment. However, I observed a few `delivered` orders with a null approval time, which points to a **data integrity issue** that must be investigated.

3.  **`order_delivered_carrier_date` (1,783 Nulls):**
    This field is null for orders that were never handed off to the shipping partner. My analysis shows these nulls are linked to 7 unique statuses (including `canceled`, `processing`, and `invoiced`), confirming these orders were stopped *before* shipment.

4.  **`order_delivered_customer_date` (2,965 Nulls):**
    This is the largest group of nulls and represents all orders that did not reach the customer. This is not limited to `canceled` orders; it correctly includes orders with `shipped`, `invoiced`, and `unavailable` statuses, which are still in transit or have failed delivery. These nulls are a direct proxy for "non-delivered" orders.

## Cleaning Payment Data

In [138]:
orders_payment_data.describe()

,payment_sequential,payment_installments,payment_value
count,103886.000000,103886.000000,103886.000000
mean,1.092679,2.853349,154.100372
std,0.706584,2.687051,217.489777
min,1.000000,0.000000,0.000000
25%,1.000000,1.000000,56.790001
50%,1.000000,1.000000,100.000000
75%,1.000000,4.000000,171.837498
max,29.000000,24.000000,13664.080078


### Let's check out how many zeros exists

In [143]:
## in payment installments
orders_payment_data[orders_payment_data['payment_installments'].isin([0])]

,order_id,payment_sequential,payment_type,payment_installments,payment_value
46982,744bade1fcf9ff3f31d860ace076d422,2,credit_card,0,58.689999
79014,1a57108394169c0b47d8f876acc9ba2d,2,credit_card,0,129.940002


In [154]:
## in payment value
orders_payment_data[orders_payment_data['payment_value'].isin([0])]

,order_id,payment_sequential,payment_type,payment_installments,payment_value
19922,8bcbe01d44d147f901cd3192671144db,4,voucher,1,0.0
36822,fa65dad1b0e818e3ccc5cb0e39231352,14,voucher,1,0.0
43744,6ccb433e00daae1283ccc956189c82ae,4,voucher,1,0.0
51280,4637ca194b6387e2d538dc89b124b0ee,1,not_defined,1,0.0
57411,00b1cb0320190ca0daa2c88b35206009,1,not_defined,1,0.0
62674,45ed6e85398a87c253db47c2d9f48216,3,voucher,1,0.0
77885,fa65dad1b0e818e3ccc5cb0e39231352,13,voucher,1,0.0
94427,c8c528189310eaa44a745b8d9d26908b,1,not_defined,1,0.0
100766,b23878b3e8eb4d25a158f57d96331b18,4,voucher,1,0.0


### Verifying Payment Insallment error
- I am confused to fill `payment_installments` with 1 as it may be a data entry error.
- But, If it is real and they consider 0 as 1 time payment then I can't set it to 1 as then I need to add 1 to every value.
- I will first investigate with `boleto` as this bank in `Brazil` just support 1 time payment.

In [158]:
orders_payment_data[orders_payment_data.payment_type == 'boleto']['payment_installments'].value_counts()

payment_installments
1    19784
Name: count, dtype: int64

In [ ]:
## checking for records where 

### do OrderID comes more then 1

In [152]:
orders_payment_data['order_id'].duplicated().sum()

np.int64(4446)

- Here minimum value 0 in `payment_installments` and `payment_value` is unusual and I need to check.
- But after checking for `boleto` I confirmed that 0 is data entry error and should be replaced to 1.
- Order ID has 4446 duplicates which is normal as customer can through different payment method for differnt items. I will group data.
- `payment_value` has some values exactly zero. Which are not suitable for analysis. I will be dropping them as these are exactly zero.

### Replacing 0 with 1 in payment installments

In [175]:
mask = orders_payment_data['payment_installments'].isin([0])
orders_payment_data[mask] = orders_payment_data[mask].replace({0:1})

### Dropping rows with 0 Payment Value

In [186]:
orders_payment_data = orders_payment_data[~(orders_payment_data['payment_value'] == 0)]

### Handling duplicate orders
- As, above I noticed that there are multiple order IDs exist, So i need to perform groupby on order ID. Because these are split payments not errors.
- I will groupby with `order_id` to get just unique IDs so I can merge with others tables easily.

In [191]:
payments_agg = orders_payment_data.groupby('order_id').agg(
    total_payment_value=('payment_value', 'sum'),
    total_payment_installments=('payment_installments', 'sum'),
    payment_method_count=('payment_type', 'nunique')
).reset_index()

# Checking the new shape
print(f"Original shape: {orders_payment_data.shape}")
print(f"New aggregated shape: {payments_agg.shape}")

Original shape: (103877, 5)
New aggregated shape: (99436, 4)


### Data Cleaning: Payments Table Anomalies & Aggregation

My inspection revealed critical anomalies that required correction to ensure financial accuracy and prepare the table for merging.

### 1. `payment_installments = 0`
I identified two transactions where `payment_installments` was 0. My investigation confirmed `boleto` (one-time) payments are always logged as `1`, proving this was a data entry error, likely on split-payment orders.

* **Action:** I imputed these two `0` values to `1`, correcting the data.

### 2. `payment_value = 0.0`
I found several transactions (including `voucher` and `not_defined` types) where the `payment_value` was exactly 0.0. These entries are data artifacts with no financial value.

* **Action:** I dropped all rows where `payment_value == 0` to remove this noise.

### 3. Payment Aggregation (Merge Preparation)
The `payments` table had multiple rows per `order_id` for split payments. To make it "merge-safe" (one row per order), I aggregated it.

* **Action:** I grouped the table by `order_id` and summed the `payment_value` to create a new `payments_agg` table with 99,436 unique orders, ready for joining.

## Cleaning Geolocation Data

### Checking duplicates in zipcode

In [192]:
geolocation_data['geolocation_zip_code_prefix'].duplicated().sum()

np.int64(981148)

- Now, this is a problem because each zipcode shares between many langitude and latitude. So, I should aggregate it by just getting mean values for both lat and lng.

In [200]:
geo_clean_data = geolocation_data.groupby('geolocation_zip_code_prefix').agg(
            geolocation_lat = ('geolocation_lat', 'mean'),
            geolocation_lng = ('geolocation_lng', 'mean')).reset_index()

In [201]:
# Check the new shape
print(f"Original shape: {geolocation_data.shape}")
print(f"New shape: {geo_clean_data.shape}")

Original shape: (1000163, 5)
New shape: (19015, 3)


## Data Cleaning: Geolocation De-duplication

My inspection of the `olist_geolocation_dataset` revealed over 980k duplicate entries for zip code prefixes, which would corrupt any merge. To create a clean, 1-to-1 lookup table, I aggregated this data.

* **Action:** I grouped the table by `geolocation_zip_code_prefix` and calculated the `mean` latitude and longitude. This created a new `geo_data_clean` table, ready for a safe merge with customer and seller data.

## Cleaning Products Table

In [205]:
## missing data check
products_data.isnull().sum()

product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

### Dropping
- I will drop rows where product physical attributes are missing as these are impossible to guess and not much useful in this analysis.
- Even the quantity of missing is too small.

In [208]:
products_data.dropna(subset=['product_weight_g'], inplace=True)

### Filling Others
- Other Columns are important specially the category name.
- If i drop them also then I will lose some sales data. So, I will impute `product_category_name` with constant term like `unknown`.
- And I will fill other column values with 0 as I don't have idea about them.

In [209]:
# Fill the null category with 'unknown'
products_data['product_category_name'].fillna('unknown', inplace=True)

# Fill the related numerical nulls with 0
products_data['product_name_lenght'].fillna(0, inplace=True)
products_data['product_description_lenght'].fillna(0, inplace=True)
products_data['product_photos_qty'].fillna(0, inplace=True)

In [210]:
## again missing data check
products_data.isnull().sum()

product_id                    0
product_category_name         0
product_name_lenght           0
product_description_lenght    0
product_photos_qty            0
product_weight_g              0
product_length_cm             0
product_height_cm             0
product_width_cm              0
dtype: int64

## Category Names Mapping
- I need to map english category names with product data so they become readable.
- I will be using left join on products data to  achieve this.

In [221]:
products_data = products_data.merge(category_names_data, on='product_category_name', how='left')

In [222]:
# it doesn't fill unknown category as it is new,so I will again impute
products_data['product_category_name_english'].fillna('unknown', inplace=True)

# drop the original column
products_data.drop('product_category_name', axis=1, inplace=True)

# Rename the new column
products_data.rename(columns={'product_category_name_english': 'product_category'}, inplace=True)

## data preview
products_data.head()

,product_id,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category
0,1e9e8ef04dbcff4541ed26657ea517e5,40,287,1,225,16,10,14,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,44,276,1,1000,30,18,20,art
2,96bd76ec8810374ed1b65e291975717f,46,250,1,154,18,9,15,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,27,261,1,371,26,4,26,baby
4,9dc1a7de274444849c219cff195d0b71,37,402,4,625,20,17,13,housewares


## Data Cleaning: Products Table

I addressed three distinct issues in the `olist_products_dataset` to prepare it for merging.

### 1. Missing Logistics Data (2 rows)

* **Decision:** Dropped 2 rows with null values for `product_weight_g` and all physical dimensions.
* **Justification:** These values are impossible to impute accurately. The row count (0.006%) is statistically insignificant and cleaner to remove.

### 2. Missing Category & Attribute Data (610 rows)

* **Decision:** Imputed these null values to retain all associated sales data.
* **Justification:** Dropping 610 products (1.85% of the catalog) would mean losing all corresponding order information.
* **Action (Category):** Filled null `product_category_name` with `'unknown'`. This preserves the products and turns "uncategorized" into a new, trackable group.
* **Action (Numerical):** Filled related nulls (`product_name_lenght`, `product_photos_qty`, etc.) with `0` as a logical default for un-cataloged items.

### 3. Category Name Translation

* **Decision:** Merged the `products` table with the `category_names_data` translation table.
* **Justification:** English category names are essential for professional reporting and the final dashboard.
* **Action:** I used a `left` merge to ensure all products, including the new `'unknown'` category, were preserved. I then dropped the original Portuguese column.

## Cleaning Sellers Table

In [223]:
## checking for seller id uniqueness
sellers_data['seller_id'].duplicated().sum()

np.int64(0)

In [225]:
sellers_data.isnull().sum()

seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64

- There is no duplicate seller.
- This table is clean and has no missing value.

## Data Cleaning: Sellers Table

* **Action:** No cleaning was required.
* **Justification:** My inspection confirmed the `olist_sellers_dataset` is 100% complete, with no null values and a unique `seller_id` for every row. This table is clean and ready for merging.

## Cleaning Reviews

In [236]:
# checking if 1 order has multiple reviews
orders_review_data.groupby('order_id')['review_id'].count().sort_values(ascending=False).isin([2,3]).sum()

np.int64(547)

- There are 547 order IDs exist which have more then 1 review.
- I will be picking most recent review as it is last from customer.

In [241]:
# converting date columns into datetime format
date_columns = ['review_creation_date', 'review_answer_timestamp']
for col in date_columns:
    orders_review_data[col] = pd.to_datetime(orders_review_data[col], errors='coerce')

In [242]:
# Sort by order_id and then date (newest first)
reviews_sorted = orders_review_data.sort_values(by=['order_id', 'review_creation_date'], ascending=[True, False])

# Drop duplicates, keeping only the first one
reviews_clean = reviews_sorted.drop_duplicates(subset='order_id', keep='first')

# Check the new shape
print(f"Original shape: {orders_review_data.shape}")
print(f"New de-duplicated shape: {reviews_clean.shape}")

Original shape: (99224, 7)
New de-duplicated shape: (98673, 7)


### Data Cleaning: Reviews Table De-duplication

My inspection revealed that 547 `order_id`s had multiple reviews (2 or 3), likely from customers updating their score or comment. Merging this table as-is would corrupt the data.

* **Decision:** I chose to keep only the **latest review** for each order, as this represents the customer's final satisfaction.
* **Action:** I sorted the table by `review_creation_date` (descending) and then used `drop_duplicates` on `order_id`, keeping the `first` (newest) entry. This creates a clean, "merge-safe" table with one unique review per order.

## Dropping Irrelevant Columns

### Listing all Tables with Columns

In [245]:
for name, data in data_objects_map.items():
    print(name)
    display(data.columns.to_list())
    print("\n")

Orders Data


['order_id',
 'customer_id',
 'order_status',
 'order_purchase_timestamp',
 'order_approved_at',
 'order_delivered_carrier_date',
 'order_delivered_customer_date',
 'order_estimated_delivery_date']



Customers Data


['customer_id',
 'customer_unique_id',
 'customer_zip_code_prefix',
 'customer_city',
 'customer_state']



GeoLocation Data


...

['product_id',
 'product_category_name',
 'product_name_lenght',
 'product_description_lenght',
 'product_photos_qty',
 'product_weight_g',
 'product_length_cm',
 'product_height_cm',
 'product_width_cm']



Sellers Data


['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state']



Category Names Data


['product_category_name', 'product_category_name_english']

### Data Cleaning: Dropping Irrelevant Columns

Before the final merge, I am dropping columns that are not relevant to main business questions (satisfaction, profitability, and RFM analysis). This will create a lighter, faster, and more focused master table.

### 1. From `reviews_clean`
* `review_id`: Redundant key; `order_id` is the join key.
* `review_comment_title` & `review_comment_message`: Out of scope for this project, as I am not performing NLP. These columns also contain many nulls.
* `review_creation_date` & `review_answer_timestamp`: Irrelevant. My analysis focuses on delivery dates, not review submission dates.

### 2. From `orders_item_data`
* `order_item_id`: A sequential ID for items within a single order. It is not needed for my aggregated analysis.
* `shipping_limit_date`: The actual delivery dates from the `orders` table are far more important than this seller-promised date.

### 3. From `products_data`
* `product_name_lenght`: Irrelevant to profitability.
* `product_description_lenght`: Irrelevant to profitability.
* `product_photos_qty`: Irrelevant to profitability.

In [249]:
products_data

,product_id,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category
0,1e9e8ef04dbcff4541ed26657ea517e5,40,287,1,225,16,10,14,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,44,276,1,1000,30,18,20,art
2,96bd76ec8810374ed1b65e291975717f,46,250,1,154,18,9,15,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,27,261,1,371,26,4,26,baby
4,9dc1a7de274444849c219cff195d0b71,37,402,4,625,20,17,13,housewares
...,...,...,...,...,...,...,...,...,...
32944,a0b7d5a992ccda646f2d34e418fff5a0,45,67,2,12300,40,40,40,furniture_decor
32945,bf4538d88321d0fd4412a93c974510e6,41,971,1,1700,16,19,16,construction_tools_lights
32946,9a7c6041fa9592d9d9ef6cfe62a71f8c,50,799,1,1400,27,7,27,bed_bath_table
32947,83808703fc0706a22e264b9d75f04a2e,60,156,2,700,31,13,20,computers_accessories


In [250]:
## Dropping these columns
# from reviews table
reviews_clean.drop(['review_id', 'review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp'], axis=1, inplace=True, errors='ignore')

# from orders items data
orders_item_data.drop(['order_item_id', 'shipping_limit_date'], axis=1, inplace=True, errors='ignore')

# from products data
products_data.drop(['product_name_lenght', 'product_description_lenght', 'product_photos_qty'], axis=1, inplace=True, errors='ignore')

# The Grand Merge

## Establishing the Core Table

I am establishing the `orders_data` table as the core of my final merged dataset. It serves as the single source of truth, as it contains all 99,441 original orders.

My merge strategy will be to **`LEFT` join** all other aggregated and cleaned data tables (e.g., `payments_agg`, `reviews_clean`, `orders_item_data`) onto this core table. This ensures I preserve every order record, even if it lacks a corresponding payment or review, which is essential for a complete analysis.

In [261]:
df_orders.shape

(99441, 8)

In [252]:
df_orders = orders_data

In [264]:
## Because order items can contain order_ids twice or more
orders_item_data['order_id'].duplicated().sum()

np.int64(13984)

## Merging Order Items (One-to-Many)

I am merging the `orders_item_data` into the core `orders` table using a `left` join.

* **Justification:** This is a **one-to-many** merge. The `orders_item_data` has 112,650 rows because many orders contain multiple items.
* **Result:** The new table shape is (113,425, 12). This is correct. It preserves all 775 orders that had no items (as NaNs) and correctly duplicates the order rows for each item they contained. My master table is now at the **item-level**.

In [262]:
orders_items_df = df_orders.merge(orders_item_data, on='order_id', how='left')

In [263]:
orders_items_df.shape

(113425, 12)

## Merging Aggregated Payments

I previously aggregated the `payments` table to prevent merge-explosions from split payments. This gave me a "merge-safe" table (`payments_agg`) with one row per `order_id`, containing `total_payment_value`, `total_payment_installments`, and `payment_method_count`.

I am now merging this `payments_agg` table (99,436 rows) into my master table using a `left` join.

* **Justification:** A `left` join is essential to preserve all 113,425 rows from my master (item-level) table. Orders that had no valid payment record will correctly show `NaN` for payment details.

In [269]:
## merging with payment_agg data.
oip_data = orders_items_df.merge(payments_agg, on='order_id', how='left')     ## combined data from orders, orders items, payment. (oip)

In [273]:
oip_data.shape

(113425, 15)

## Merging Cleaned Reviews (One-to-One)

I previously de-duplicated the `reviews` table by keeping only the latest review for each `order_id`. This `reviews_clean` table is now "merge-safe" for a one-to-one join.

* **Action:** I am merging this `reviews_clean` table into my master DataFrame using a `left` join.
* **Justification:** The shape of the master table remained unchanged (113,425 rows). This is the expected and correct result, as it confirms that each unique order item is now successfully mapped to its single, most recent review score. Orders without a review are correctly preserved with `NaN` values.

In [283]:
oip_reviews_data = oip_data.merge(reviews_clean, on='order_id', how='left')

## Merging Customer Information (One-to-One)

I merged the `customers_data` table into the master DataFrame using `customer_id` as the key.

* **Verification:** I confirmed key integrity using `np.intersect1d` before the merge.
* **Justification:** The total row count remained unchanged (113,425). This is the correct and expected result, as it successfully maps customer-specific details (like `customer_unique_id` and `customer_state`) to each order item.

In [282]:
## check if any customer id in orders exist in customers data
np.intersect1d(opi_reviews_data['customer_id'], customers_data['customer_id']).shape

(99441,)

In [285]:
opir_customers_df = oip_reviews_data.merge(customers_data, on='customer_id', how='left')

## Merging Product Information

I merged the cleaned `products_data` (containing 32,949 unique products) into the master table using `product_id` as the key.

* **Verification:** I first ran an `inner` join test, which showed that 18 `product_id`s from the original `order_items` table were missing from the `products` catalog. This is a minor data integrity gap (0.016%) and acceptable.
* **Justification:** I used a `left` join for the final merge. This is essential to preserve all 113,425 order-item rows. The 18 un-cataloged items will correctly show `NaN` for product details (category, weight, etc.), which is the expected behavior.

In [291]:
orders_item_data.merge(products_data, on='product_id').shape

(112632, 10)

In [294]:
merged_data = opir_customers_df.merge(products_data, on='product_id', how='left')

In [296]:
merged_data.shape

(113425, 25)

## Merging Seller Information

I merged the `sellers_data` table into the master DataFrame using `seller_id` as the key.

* **Justification:** This final merge links each order item to its specific seller, allowing for seller-based performance and location analysis.
* **Verification:** The merge completed successfully. My analysis confirmed that 775 rows now show a `NaN` for `seller_id`. This is the **correct and expected outcome**, as these 775 rows are the original orders that had no items, and therefore, no associated seller.

In [299]:
final_data = merged_data.merge(sellers_data, on='seller_id', how='left')

In [300]:
final_data['seller_id'].isnull().sum()

np.int64(775)

## Validation Checks

### Unique OrderID count

In [307]:
if (orders_data['order_id'].nunique() == final_data['order_id'].nunique()):
    print("OrderID is unique and has same count.")
else:
    print("OrderID is not unique.")

OrderID is unique and has same count.


### Nulls in ID cols

In [314]:
print(final_data['order_id'].isnull().sum())
print(final_data['product_id'].isnull().sum())
print(final_data['seller_id'].isnull().sum())

0
775
775


### Check for Logical Dates

In [317]:
print((final_data['order_delivered_customer_date'] < final_data['order_purchase_timestamp']).sum())

0


### Check for "Merge Explosion" Duplicates

In [322]:
print(final_data.duplicated().sum())

10225


In [323]:
## renaming dataframe
df_master = final_data

## Final Validation: Master Table Sanity Check

Before proceeding to analysis, I performed a final validation to ensure the merged master table is structurally sound.

* **1. `order_id` Null Check:** `df_master['order_id'].isnull().sum()`
    * **Result:** 0.
    * **Finding:** This is perfect. My primary key is 100% intact, and every row is tied to an order.

* **2. Expected Nulls Check:** `df_master['product_id'].isnull().sum()`
    * **Result:** 775.
    * **Finding:** This is also correct. It confirms that the 775 orders that had no items (and thus no `product_id` or `seller_id`) were successfully preserved by my `left` join.

* **3. Logical Date Check:** `(df_master['order_delivered_customer_date'] < df_master['order_purchase_timestamp']).sum()`
    * **Result:** 0.
    * **Finding:** This confirms there are no impossible date patterns (e.g., delivery before purchase), and my `datetime` merges were successful.

* **4. Full Row Duplicates Check:** `df_master.duplicated().sum()`
    * **Result:** 10,225.
    * **Finding:** This is not an error but the **expected outcome**. These duplicates represent orders where a customer bought **multiple identical items** (e.g., 2 of the same t-shirt). Because I dropped `order_item_id`, these rows are now identical. Dropping them would be a mistake, as it would cause me to under-report `price` and `freight_value` in my profitability analysis.

---

## Merge Strategy Justification (Why `orders` + `LEFT` Joins)

This is a critical documentation note on my methodology.

### 1. Why the `orders_data` Table as the Core?

I chose the `orders` table as the "core" because it represents the **single source of truth for all transactions**.

* It contains the complete, 99,441-record history of every order placed, regardless of its status (e.g., delivered, canceled, or in-progress).
* Every key business question—"Was the customer satisfied?", "Was the order profitable?"—originates from an `order_id`. Starting with this table ensures my analysis is built on the complete and original customer record.

### 2. Why Use `LEFT` Joins Exclusively?

My decision to use only `LEFT` joins (starting from the `orders` table) was deliberate to **preserve the integrity of the full dataset.**

* **An `inner` join would hide the truth.** For example, an `inner` join with the `reviews` table would *drop* all orders that were never reviewed. This would make it impossible to answer questions like, "Do canceled orders get reviewed?"
* **`LEFT` joins turn "missing data" into an insight.** By using `LEFT` joins, orders that were canceled (and thus have no `order_delivered_customer_date`) are kept, appearing as `NaN`. This allows me to analyze them as a specific cohort. If I had used an `inner` join, these orders would have simply vanished, leading to a biased and incorrect analysis.

This "keep everything" strategy ensures I am analyzing the *complete* business picture, including the failures and incomplete records, which are often the most important.

## Phase 1 Complete: Master Table Created

All 9 data tables have been cleaned, optimized, and merged into a single, item-level master DataFrame.

* **Final Shape:** (113,425, 28)
* **Validation:** All integrity checks (nulls, logical dates, duplicates) have been performed and passed.
* **Next Step:** I will now save this final DataFrame to `outputs/master_data.csv` and proceed to a new notebook (`02_Analysis.ipynb`) for feature engineering and exploratory data analysis.